# Analysis of intraday storage

Using the base case hydrology and P&E modelling, determine the maximum volume of water that can be held back during two 10 hour storage periods and then released during two 2 hour peaking periods. 

Calculate the energy transferred from the storage (off-peak) to peak periods.

Calculate the rise in the headpond and tailrace during each storage period (and therefore fall during each peak period). 

In [431]:
flow_data='./output_data/ngonye_daily.csv'
energy_data='./output_data/base_pe_daily.csv'

canal_capacity=1100
rated_flow_unit=250

#offpeak_min_flow=rated_flow_unit * 0.5
#offpeak_min_flow=50
offpeak_min_flow=rated_flow_unit

storage_curve=10 # 10 Mm3/m headpond rise - From Feasibility Study Rev D Appendix H - Mott MacDonald
max_power=185 # Max output for whole plant
fixed_tailwater_level = 0

peak_hours = 3 # Length of morning and evening peaks
store_hours = 7 # Time between morning peak and evening peak when storing can occur

In [432]:
import numpy as np
import pandas as pd
import datetime
import math

In [433]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [434]:
flow = pd.read_csv(flow_data).set_index('Date')
flow.columns

Index(['LaggedDate', 'VicFalls', 'Conversion', 'Flow', 'Exceedance', 'Year',
       'Month', 'Day', 'MonthId', 'WaterYear', 'WaterMonth', 'WaterDay',
       'WaterWeek', 'Volume', 'Flow_difference', 'Flow_difference_pct',
       'EWRRefExceedance'],
      dtype='object')

In [435]:
pe = pd.read_csv(energy_data).set_index('Date')
pe.columns

Index(['LaggedDate', 'VicFalls', 'Conversion', 'Flow', 'Exceedance', 'Year',
       'Month', 'Day', 'MonthId', 'WaterYear', 'WaterMonth', 'WaterDay',
       'WaterWeek', 'Volume', 'Flow_difference', 'Flow_difference_pct',
       'EWRRefExceedance', 'EWRBandNo', 'EWRBandLabel', 'EWRChannelA',
       'EWRChannelC', 'EWRChannelD', 'EWRChannelE', 'EWRChannelFG', 'EWRTotal',
       'EWRProportion', 'FlowAvailableForGeneration', 'FlowCanal',
       'LowFlowShutoff', 'Turbines', 'FlowTurbine1', 'FlowTurbine2',
       'FlowTurbine3', 'FlowTurbine4', 'FlowSpill', 'SpillChannelA',
       'SpillChannelC', 'SpillChannelD', 'SpillChannelE', 'SpillChannelFG',
       'FlowChannelA', 'FlowChannelC', 'FlowChannelD', 'FlowChannelE',
       'FlowChannelFG', 'FlowLeftChannel', 'LevelTailwater', 'LevelHeadpond',
       'HeadlossLeftChannel', 'HeadlossCanal', 'LevelForebay',
       'HeadlossTurbine1', 'HeadlossTurbine2', 'HeadlossTurbine3',
       'HeadlossTurbine4', 'HeadTurbine1', 'HeadTurbine2', 'HeadTur

In [436]:

effective_efficiency = pe['EffTurbine1'].quantile(0.8)*pe['EffGen1'].quantile(0.8)

In [437]:
work=pe[['LowHeadShutoff','EWRTotal','FlowSpill','FlowAvailableForGeneration','FlowCanal','Power','Energy','LevelHeadpond','LevelTailwater', 'HeadTurbine1']].copy()
work['Flow']=flow['Flow']
work['Exceedance']=flow['Exceedance']

work['LowHeadShutoff'] = work['LowHeadShutoff'].fillna(0)
work.columns

Index(['LowHeadShutoff', 'EWRTotal', 'FlowSpill', 'FlowAvailableForGeneration',
       'FlowCanal', 'Power', 'Energy', 'LevelHeadpond', 'LevelTailwater',
       'HeadTurbine1', 'Flow', 'Exceedance'],
      dtype='object')

In [438]:
#Total generation volume Mm3
work['GenerationVol'] = work['FlowCanal'] * (24 * 60 * 60) / 1e+06

#Rough calculation of the maximum flow during peak time - flow that produced maximum output at current head and high efficiency
work['MaxGenerationFlow'] = max_power / work['HeadTurbine1'] / 9.81 / effective_efficiency * 1000
#Limit maximum flow to canal capacity
work['MaxGenerationFlow'] = np.minimum(work['MaxGenerationFlow'],canal_capacity)

#Maximum flow that could be given to the peak from storage m3/s
work['MaxFlowFromStore'] = work['MaxGenerationFlow'] - work['FlowCanal']

#Max volume per cycle that could be given to the peak Mm3
work['MaxVolFromStore'] = work['MaxFlowFromStore'] * (peak_hours * 60 * 60) / 1e+06

In [439]:
#Max volume per cycle that could be taken from the offpeak for storage Mm3
work['MaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * ((12 - peak_hours) * 60 * 60) / 1e+06).clip(lower=0)
#Volume per cycle that will be stored (min of available or needed)
work['StoreVol'] = work[['MaxVolFromStore','MaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'StoreVol'] = 0

work.loc[work['StoreVol'] == work['MaxVolFromStore'], 'FullPeak'] = 'True'
work.loc[work['StoreVol'] < work['MaxVolFromStore'], 'FullPeak'] = 'False'

In [440]:
work['PeakStoreFlow']=(work['StoreVol'] / (60 * 60 * peak_hours / 1e+06)) # Extra flow from storage during peaks 
work['OffpeakStoreFlow']=(work['StoreVol'] / (60 * 60 * (12 - peak_hours) / 1e+06)) # Flow diverted to storage during offpeak

work['OffpeakFlowCanal']=work['FlowCanal'] - work['OffpeakStoreFlow'] #Resulting canal flow during off-peak
work['PeakFlowCanal']=work['FlowCanal'] + work['PeakStoreFlow'] #Resulting canal flow during peak

work['PeakVol']=work['PeakFlowCanal'] * (peak_hours * 2) * 60 * 60 / 1e+06 #Volume per day (2 cycles) for the peaks Mm3
work['OffpeakVol']=work['OffpeakFlowCanal'] * (24 - (peak_hours * 2)) * 60 * 60 / 1e+06 #Volume per day (2 cycles) for the offpeaks Mm3

In [441]:
work['PeakEnergy'] = work['Energy']/work['GenerationVol']*work['PeakVol'] #Daily energy during the peaks 
work['OffpeakEnergy'] = work['Energy']/work['GenerationVol']*work['OffpeakVol'] #Daily energy during the off-peaks
work['EnergyTransfer'] = work['Energy']/work['GenerationVol']*(work['StoreVol']*2) #Ammount of energy moved from off-peak to peak by storage
work['EnergyTransferProp'] = work['EnergyTransfer']/work['Energy']

work['PeakPower'] = work['PeakEnergy'] / (peak_hours * 2) #Power output during the peak
work['OffpeakPower'] = work['OffpeakEnergy'] / (24 - (peak_hours * 2)) #Power output during the off-peak

In [442]:
work['HeadpondDrop']=work['StoreVol']/storage_curve # meters of rise/drop at the headpond during each cycle
work['PeakTailwaterFlow']=work['FlowSpill'] + work['EWRTotal'] + work['PeakFlowCanal'] # Flow at the tailwater during peak
work['OffpeakTailwaterFlow']=work['FlowSpill'] + work['EWRTotal'] + work['OffpeakFlowCanal'] # Flow at the tailwater during offpeak
work['PeakTailwaterLevel']=work['PeakTailwaterFlow'].apply(tailwaterLevel) # Level at the tailwater at peak
work['OffpeakTailwaterLevel']=work['OffpeakTailwaterFlow'].apply(tailwaterLevel) # Level at the tailwater at offpeak
work['TailwaterDrop']=work['PeakTailwaterLevel'] - work['OffpeakTailwaterLevel']



In [443]:
work['ExceedanceRnd']=np.round(work['Exceedance']*2,2)/2
fdc=work.reset_index().groupby('ExceedanceRnd').mean().drop(['LowHeadShutoff'],axis=1)
fdc

,EWRTotal,FlowSpill,FlowAvailableForGeneration,FlowCanal,Power,Energy,LevelHeadpond,LevelTailwater,HeadTurbine1,Flow,...,EnergyTransfer,EnergyTransferProp,PeakPower,OffpeakPower,HeadpondDrop,PeakTailwaterFlow,OffpeakTailwaterFlow,PeakTailwaterLevel,OffpeakTailwaterLevel,TailwaterDrop
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,276.632353,8588.543600,8588.543600,0.000000,0.000000,0.000000,990.527642,981.764386,6.826354,8865.175953,...,NaN,NaN,NaN,NaN,0.0,8865.175953,8865.175953,981.764386,981.764386,0.0
0.005,324.517241,6154.272848,6154.272848,0.000000,0.000000,0.000000,990.128209,979.484350,8.706957,6478.790089,...,NaN,NaN,NaN,NaN,0.0,6478.790089,6478.790089,979.484350,979.484350,0.0
0.010,321.165680,4741.890222,5223.547027,481.656805,38.471131,923.307133,990.050824,978.148915,9.965006,5544.712707,...,0.0,0.0,87.859744,87.859744,0.0,5544.712707,5544.712707,978.148915,978.148915,0.0
0.015,305.947674,3785.016338,4885.016338,1100.000000,91.548338,2197.160113,990.022062,977.618191,10.466969,5190.964012,...,0.0,0.0,91.548338,91.548338,0.0,5190.964012,5190.964012,977.618191,977.618191,0.0
0.020,302.523256,3375.069915,4475.069915,1100.000000,98.867101,2372.810412,990.000000,976.964195,11.098903,4777.593171,...,0.0,0.0,98.867101,98.867101,0.0,4777.593171,4777.593171,976.964195,976.964195,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,64.833333,0.000000,120.810563,120.810563,26.263488,630.323719,990.000000,964.428484,25.372253,185.643897,...,0.0,0.0,26.263488,26.263488,0.0,185.643897,185.643897,964.428484,964.428484,0.0
0.985,60.697115,0.000000,114.563550,114.563550,24.754930,594.118324,990.000000,964.449454,25.365246,175.260665,...,0.0,0.0,24.754930,24.754930,0.0,175.260665,175.260665,964.449454,964.449454,0.0
0.990,56.600000,0.000000,108.425295,108.425295,23.264526,558.348614,990.000000,964.467851,25.359957,165.025295,...,0.0,0.0,23.264526,23.264526,0.0,165.025295,165.025295,964.467851,964.467851,0.0


In [444]:
fdc.loc[0.75]

EWRTotal                       117.621399
FlowSpill                        0.000000
FlowAvailableForGeneration     206.167896
FlowCanal                      206.167896
Power                           45.418595
Energy                        1090.046278
LevelHeadpond                  990.000000
LevelTailwater                 964.970663
HeadTurbine1                    24.585224
Flow                           323.789296
Exceedance                       0.750852
GenerationVol                   17.812906
MaxGenerationFlow              832.680880
MaxFlowFromStore               626.512984
MaxVolFromStore                  6.766340
MaxVolToStore                    0.006635
StoreVol                         0.006635
PeakStoreFlow                    0.614349
OffpeakStoreFlow                 0.204783
OffpeakFlowCanal               205.963114
PeakFlowCanal                  206.782245
PeakVol                          4.466496
OffpeakVol                      13.346410
PeakEnergy                     273

In [445]:
work.to_csv('./intraday/intraday_daily_1unit.csv')

In [446]:
fdc.to_csv('./intraday/intraday_fdc_1unit.csv')